# interpolate UFS grid to observation grid

In [1]:
import xarray as xr
import numpy as np

In [2]:
# ufs path
ufs_path = '/scratch/hbai/ufs_p6_daily/z500/'

# obeservation path
obs_path = '/scratch/hbai/gpcp/z500_31day_ufsexp/'

In [3]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [4]:
# create a list of ufs files to open
# this example shows how to open on
ufs_files = [f'{ufs_path}ufs_p6_2012{month}01_z500.nc' for month in months]
ufs_files

['/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120101_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120201_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120301_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120401_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120501_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120601_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120701_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120801_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120901_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121001_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121101_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121201_z500.nc']

In [5]:
# read one year's ufs files at once
ds = xr.open_mfdataset(ufs_files, combine='nested', concat_dim='month')
z500_ufs = ds.z500
z500_ufs

<xarray.DataArray 'z500' (month: 12, ncl0: 31, latitude: 721, longitude: 1440)>
dask.array<concatenate, shape=(12, 31, 721, 1440), dtype=float64, chunksize=(1, 31, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Dimensions without coordinates: month, ncl0
Attributes:
    average_op_ncl:  dim_avg_n function was applied

In [6]:
# read target lat&lon from observation
ds2 = xr.open_dataset(obs_path+"erai.z500.mean.20120101.nc")
lat = ds2.lat
lon = ds2.lon

In [7]:
# interpolate ufs to grid of observation
z500_ufs_int = z500_ufs.interp(latitude=lat, longitude=lon, method='linear')
z500_ufs_int

<xarray.DataArray 'z500' (month: 12, ncl0: 31, lat: 256, lon: 512)>
dask.array<dask_aware_interpnd, shape=(12, 31, 256, 512), dtype=float64, chunksize=(1, 31, 256, 512), chunktype=numpy.ndarray>
Coordinates:
    latitude   (lat) float32 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
    longitude  (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * lat        (lat) float32 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float32 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: month, ncl0
Attributes:
    average_op_ncl:  dim_avg_n function was applied

In [8]:
# write interpolated ufs data into nctcdf files
for i, month in enumerate(months):
    z500_ufs_int[i].to_netcdf(ufs_path+"ufs_p6_2012"+month+"01_z500_interp.nc")
    